<a href="https://colab.research.google.com/github/nh7881/machine_learning_project_class/blob/master/exercise_10/assignment_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
from torch import nn
from torchvision import transforms, datasets
from matplotlib import pyplot as plt
import numpy as np

In [19]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

In [20]:
data_path = './MNIST'
minibatch_size=64

training_set = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)
testing_set = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
training_loader = torch.utils.data.DataLoader(training_set,minibatch_size)
testing_loader = torch.utils.data.DataLoader(testing_set,minibatch_size)

In [21]:
print(training_loader.dataset.data.shape)
print(testing_loader.dataset.data.shape)

torch.Size([10000, 28, 28])
torch.Size([60000, 28, 28])


In [27]:
class classification(nn.Module):
    def __init__(self):
        super(classification, self).__init__()
        
        # construct layers for a neural network
        self.classifier1 = nn.Sequential(
            nn.Linear(in_features=28*28, out_features=20*20),
            nn.ReLU(),
            nn.Dropout2d(0.2),
        ) 
        self.classifier2 = nn.Sequential(
            nn.Linear(in_features=20*20, out_features=10*10),
            nn.ReLU(),
            nn.Dropout2d(0.2),
        ) 
        self.classifier3 = nn.Sequential(
            nn.Linear(in_features=10*10, out_features=10),
            nn.LogSoftmax(dim=1),
        ) 
        
        
    def forward(self, inputs):                 # [batchSize, 1, 28, 28]
        x = inputs.view(inputs.size(0), -1)    # [batchSize, 28*28]
        x = self.classifier1(x)                # [batchSize, 20*20]
        x = self.classifier2(x)                # [batchSize, 10*10]
        out = self.classifier3(x)              # [batchSize, 10]
        
        return out


In [28]:
criterion = nn.NLLLoss()

In [29]:
learning_rate_value = 0.01
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
classifier = classification().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=learning_rate_value, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)

In [30]:
test_losses = []
test_accuracies = []
def test() :
    classifier.eval()
    correct = 0
    n = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(testing_loader):
            data, target = data.to(device), target.to(device)
            output = classifier(data)
            loss = criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            n += len(data)
        print('Test Loss: {:.6f} Correct: {}, ACC : {}'.format(
            loss.item(), correct, correct/n))
        test_losses.append(loss.item())
        test_accuracies.append(correct/n)

In [ ]:
epoch_num = 30
losses = []
accuracies = []
for epoch in range(epoch_num) :
    correct = 0
    n = 0
    classifier.train()
    #scheduler.step()
    for batch_idx, (data, target) in enumerate(training_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = classifier(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        n += len(data)
    print('Train Epoch: {} Loss: {:.6f} Correct: {} ACC: {}'.format(
        epoch, loss.item(), correct, correct/n))
    test()
    losses.append(loss.item())
    accuracies.append(correct/n)
    #train  test    batch_size  loss    activ
    #94.7,  91.95   16          ce      sigmoid
    #?      93.7    64          ce      sigmoid
    #97     92.8    64          nll     sigmoid
    #100    94.9    16          nll     sigmoid
    #100    95.5    64          nll     relu
    #100    95.5    64          nll     relu    dropout추가

Train Epoch: 0 Loss: 0.292078 Correct: 7412 ACC: 0.7412
Test Loss: 0.151410 Correct: 52036, ACC : 0.8672666666666666
Train Epoch: 1 Loss: 0.081110 Correct: 9026 ACC: 0.9026
Test Loss: 0.085904 Correct: 54441, ACC : 0.90735
Train Epoch: 2 Loss: 0.050230 Correct: 9266 ACC: 0.9266
Test Loss: 0.040112 Correct: 55515, ACC : 0.92525
Train Epoch: 3 Loss: 0.014734 Correct: 9455 ACC: 0.9455
Test Loss: 0.014855 Correct: 55882, ACC : 0.9313666666666667
Train Epoch: 4 Loss: 0.022258 Correct: 9563 ACC: 0.9563
Test Loss: 0.009833 Correct: 56339, ACC : 0.9389833333333333
Train Epoch: 5 Loss: 0.011892 Correct: 9632 ACC: 0.9632
Test Loss: 0.007022 Correct: 56535, ACC : 0.94225
Train Epoch: 6 Loss: 0.003576 Correct: 9720 ACC: 0.972
Test Loss: 0.007344 Correct: 56709, ACC : 0.94515
Train Epoch: 7 Loss: 0.002484 Correct: 9764 ACC: 0.9764
Test Loss: 0.004359 Correct: 56922, ACC : 0.9487
Train Epoch: 8 Loss: 0.001277 Correct: 9819 ACC: 0.9819
Test Loss: 0.003604 Correct: 57042, ACC : 0.9507
Train Epoch: 9 L